In [1]:
from simpletransformers.config.model_args import ModelArgs
from simpletransformers.language_representation import RepresentationModel
from transformers import BertTokenizer

import os
import time
import string
import torch
import pickle
import itertools
import numpy as np
import sys
import importlib
import pathlib
import itertools
from tqdm import tqdm
from collections import OrderedDict
import re
import torch.nn as nn
import torch.optim as optim
from sklearn.neighbors import KNeighborsClassifier
from typing import List, Tuple
import torch.nn.functional as F
import torch.autograd as autograd
from torch.functional import Tensor
from transformers import AutoTokenizer, BertTokenizer
from gensim.models import Word2Vec, KeyedVectors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPClassifier
from torch.utils.data import sampler, DataLoader, TensorDataset
from typing import Dict, List, Sequence, Set, Text, Tuple, Union
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, fbeta_score, accuracy_score
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_validate
from imblearn.metrics import geometric_mean_score 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from simpletransformers.config.model_args import ModelArgs
from simpletransformers.language_representation import RepresentationModel
from transformers import BertTokenizer

def pickle_data(data, file_name):
    with open(file_name, 'wb') as file:
        pickle.dump(data, file)
        
def unpickle_data(data):
    with open(data, "rb") as file:
        loaded = pickle.load(file)
    return loaded

def reader(root: str, file: str):
    with open(os.path.join(root, file), "r") as f: 
        return f.readlines()

def json_r(path):
    with open(path, "r") as file:
        loaded = json.load(file)
    return loaded

### Guided Vocabulary

In [15]:
# load raw files

# get train, dev and test sets
d2a_train = pd.read_csv("/Users/Gabriel/Projects/vReduce/data/d2a/function/train.csv")
d2a_dev = pd.read_csv("/Users/Gabriel/Projects/vReduce/data/d2a/function/train.csv")
d2a_test = pd.read_csv("/Users/Gabriel/Projects/vReduce/data/d2a/function/train.csv")
d2a_train_files, d2a_dev_files, d2a_test_files = list(d2a_train["code"]), list(d2a_dev["code"]), list(d2a_test["code"]), 
d2a_train_labels, d2a_dev_labels = np.array(list(d2a_train["label"])), np.array(list(d2a_dev["label"]))

# get asts but first compile c/c++ code
def get_asts(project):
    out = {}
    for root, _, files in os.walk(project):
        for file in files:
            if file.endswith(".txt"):
                source_file_pth = os.path.join(root, file) 
                source_file_name = int(pathlib.Path(source_file_pth).stem.split(".")[0])
                try:
                    with open(source_file_pth, "r") as source_file:
                        out[source_file_name] = source_file.read()
                except OSError as e:
                    raise e
    return list(OrderedDict(sorted(out.items())).values())

# retrieve extracted asts
d2a_train_asts = get_asts("/Users/Gabriel/Projects/vreduce/data/d2a/raw/train")
d2a_dev_asts = get_asts("/Users/Gabriel/Projects/vreduce/data/d2a/raw/dev")
d2a_test_asts = get_asts("/Users/Gabriel/Projects/OUR/data/d2a/raw/test")

# get features 
def extract_features(observations, features): # main
    nodes = []
    for observation in observations:
        temp = []
#         temp = data.split()
        for data in observation.split():
            if data.endswith(features):
                temp.append(data)
        nodes.append(temp)
    out = []
    for node in nodes:
        temp = []
        for feature in node:
            temp.append(feature.translate(str.maketrans("", "", string.punctuation)))
        out.append(temp)
    return out

# d2a train features
d2a_stmts, d2a_exprs, d2a_decls = extract_features(d2a_train_asts, "Stmt"), extract_features(d2a_train_asts, "Expr"), extract_features(d2a_train_asts, "Decl")
d2a_train_features = (d2a_stmts, d2a_exprs, d2a_decls)
pickle_data(d2a_train_features, "/Users/Gabriel/Projects/vReduce/vreduce/selected/d2a/d2a_train_features.pkl")


d2a_stmts, d2a_exprs, d2a_decls = extract_features(d2a_dev_asts, "Stmt"), extract_features(d2a_dev_asts, "Expr"), extract_features(d2a_dev_asts, "Decl")
d2a_dev_features = (d2a_stmts, d2a_exprs, d2a_decls)
pickle_data(d2a_dev_features, "/Users/Gabriel/Projects/vReduce/vreduce/selected/d2a/d2a_dev_features.pkl")

d2a_stmts, d2a_exprs, d2a_decls = extract_features(d2a_test_asts, "Stmt"), extract_features(d2a_test_asts, "Expr"), extract_features(d2a_test_asts, "Decl")
d2a_test_features = (d2a_stmts, d2a_exprs, d2a_decls)
pickle_data(d2a_test_features, "/Users/Gabriel/Projects/vReduce/vreduce/selected/d2a/d2a_test_features.pkl")

In [16]:
d2a_stmts, d2a_exprs, d2a_decl = d2a_train_features


3

In [2]:
# d2a_train_1 = unpickle_data("/Users/Gabriel/Projects/vReduce/vreduce/features/d2a/one.pkl")
# d2a_train_2 = unpickle_data("/Users/Gabriel/Projects/vReduce/vreduce/features/d2a/two.pkl")
# d2a_train_3 = unpickle_data("/Users/Gabriel/Projects/vReduce/vreduce/features/d2a/three.pkl")
# d2a_train_4 = unpickle_data("/Users/Gabriel/Projects/vReduce/vreduce/features/d2a/four.pkl")
# d2a_train_5 = unpickle_data("/Users/Gabriel/Projects/vReduce/vreduce/features/d2a/five.pkl")

# # Get Features

In [4]:
d2a_train_1.shape

(1000, 768)